*SPCS Networking Connectivity Test: Microsoft Sharepoint*

1. Please provide your Sharepoint URL
2. Check that the Network Rule structure matches your requirements
3. Run the Connectivity Test if your EAI is already configured
4. Use the Cells to prepare the EAI if needed, then rerun the test.

In [ ]:
# SharePoint Connectivity Test Configuration
# Update the SHAREPOINT_SITE_URL with your actual SharePoint site URL

# User Configuration - UPDATE THIS
SHAREPOINT_SITE_URL = "https://abc.sharepoint.com/sites/xyz"  # Replace with your actual SharePoint URL
MICROSOFT_LOGIN_URL = "https://login.microsoft.com"
MICROSOFT_GRAPH_URL = "https://graph.microsoft.com"

print(f"Configuration:")
print(f"SharePoint Site: {SHAREPOINT_SITE_URL}")
print("\nReady to test connectivity...")

In [ ]:
import requests
from urllib.parse import urlparse

def test_connectivity(url, description):
    """Test HTTP connectivity to a given URL"""
    try:
        print(f"\n🔍 Testing {description}: {url}")

        # Simple HTTP test - let EAI handle the routing
        response = requests.get(url, timeout=10, allow_redirects=True)
        print(f"  ✅ HTTP Response: {response.status_code}")

        # Any response means connectivity is working
        return True

    except requests.exceptions.ConnectionError as e:
        print(f"  ❌ Connection Error: {e}")
        return False
    except requests.exceptions.Timeout as e:
        print(f"  ❌ Timeout Error: {e}")
        return False
    except Exception as e:
        print(f"  ❌ Error: {e}")
        return False

# Run connectivity tests
print("=" * 60)
print("SHAREPOINT CONNECTIVITY TEST")
print("=" * 60)

results = {}
results['sharepoint'] = test_connectivity(SHAREPOINT_SITE_URL, "SharePoint Site")
results['login'] = test_connectivity(MICROSOFT_LOGIN_URL, "Microsoft Login")
results['graph'] = test_connectivity(MICROSOFT_GRAPH_URL, "Microsoft Graph API")

print("\n" + "=" * 60)
print("CONNECTIVITY TEST SUMMARY")
print("=" * 60)

all_passed = True
for service, passed in results.items():
    status = "✅ PASS" if passed else "❌ FAIL"
    print(f"{service.upper()}: {status}")
    if not passed:
        all_passed = False

if all_passed:
    print(f"\n🎉 All connectivity tests PASSED!")
    print("Your SPCS environment can reach all required SharePoint endpoints.")
    print("You can proceed with SharePoint integration.")
else:
    print(f"\n⚠️  Some connectivity tests FAILED!")
    print("You need to configure Snowflake External Access Integration (EAI).")
    print("Please run the EAI setup cells below to resolve connectivity issues.")

If Connectivity testing fails, you can use the Cells below to prepare and implement an EAI suitable for Sharepoint access.

In [ ]:
-- Create Network Rules for SharePoint connectivity
-- Run this cell if connectivity tests failed

-- Network rule for Microsoft login services
CREATE OR REPLACE NETWORK RULE sharepoint_services_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('login.microsoftonline.com:443', 'login.microsoft.com:443', 'www.office.com:443','graph.microsoft.com:443', 'developer.microsoft.com:443');

-- Network rule for SharePoint sites (modify domain as needed)
-- You may need to update this with your specific SharePoint domain
CREATE OR REPLACE NETWORK RULE sharepoint_site_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('*.sharepoint.com:443');

SHOW NETWORK RULES LIKE 'sharepoint_%';

In [ ]:
-- Create External Access Integration for SharePoint
-- This combines all the network rules into a single integration

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION sharepoint_eai
  ALLOWED_NETWORK_RULES = (
    sharepoint_services_rule,
    sharepoint_site_rule
  )
  ENABLED = TRUE
  COMMENT = 'External Access Integration for Microsoft SharePoint connectivity';

-- Grant usage on the integration to your role
GRANT USAGE ON INTEGRATION sharepoint_eai TO ROLE ACCOUNTADMIN;

SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'sharepoint_eai';

In [ ]:
ALTER NOTEBOOK SPCSCONTEST_SHAREPOINT
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('SHAREPOINT_EAI');

After Creating and Setting the EAI on the Notebook, run the Connectivity test again.